# Bought By Many Interview Exercise

In [275]:
import os
import json
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime

%matplotlib inline

## 1. Process input files into CSVs

First, let's take a look at some policies to see what we're dealing with.

In [276]:
data_folder = 'data/policies'

def read_policy(filepath):
    
    with open(filepath) as file:
        text = file.read()
    
    return json.loads(text)

In [277]:
example = read_policy(f'{data_folder}/{os.listdir(data_folder)[0]}')

print(json.dumps(example, indent=2))

{
  "_id": "5d1cd832cd9d4f5ef38efdef",
  "created_at": 1562171442321,
  "updated_at": 1578003116164,
  "account_ref": "BBM",
  "cessation_date": 1593734400000,
  "data": {
    "insured_entities": [
      {
        "breed": "CHIHUAHUA",
        "breed_friendly": "Chihuahua",
        "dob": 1557273600000,
        "gender": "MALE",
        "healthy": true,
        "microchipped": true,
        "pedigree": false,
        "pedigree_type": "CROSS_BREED",
        "pet_address_postcode": "CV6",
        "pet_name": "Alfie",
        "photo": "https://bbm-user-data-prod.s3.amazonaws.com/96993249-9147-47eb-9d80-22c561a1d5fd/pet-photos/a17ef4c4-839e-46ba-9b0d-67fb1c7d89fb/ADB8F502-A44A-414E-98AC-DE6FFB678BCD-3p4dz.jpeg",
        "ref": "332924",
        "spayed_neutered": true,
        "species": "DOG",
        "uuid": "8aeef6e5-d182-4002-94d9-02f31c357760",
        "value": 450
      }
    ],
    "payment": {
      "direct_debit_account_details": "gAAAAABdHNgjMb8lKC5AoOX_wdLt3y1Jj3vRPg5jBKJYQSWvqK

We see a lot of information, including details of:

* the policy holder
* the insured pets
* the insurance products included in the policy (allowances, pricing, rules etc.)
* payments
* event timelines (on risk, started, cancelled etc.)

We'll want the following in order to answer the questions in part 2:
* the country and date of each policy sold
* the species and breed of each pet
* the price of each product

We'll also want to relate policies to the pet(s) covered. Although unlikely, this has the potential to be a many-to-many relationship, since each policy can cover many pets, and it might be possible for each pet can be covered by multiple policies.

Similarly, we need to model the many-to-many relationship between policies and products. A policy's sales value is determined by the prices of the products included.

In [278]:
def select(dictionary, keys) -> dict:
    '''Filters a dictionary down to only the desired keys'''
    
    return {key: value for key, value in dictionary.items() if key in keys}

In [279]:
policies, pets, mapping = [], [], []

# read each policy file one by one and extract relevant data
for filename in os.listdir(data_folder):
    policy = read_policy(f'{data_folder}/{filename}')
    policy_id = policy.get('uuid')
    
    # calculate total price of products included in policy
    price = 0
    
    if 'products' in policy:
        
        for product in policy['products']:
            price += product['price']['annual']['amount']
    
    # extract relevant policy fields
    simple_policy = select(policy, {'uuid', 'account_ref', 'created_at'})
    simple_policy['price'] = price
    policies.append(simple_policy)
    
    # extract pets covered in policy
    if 'data' in policy and 'insured_entities' in policy['data']:
        covered_pets = policy['data']['insured_entities']

        for pet in covered_pets:
            pet_id = pet.get('uuid')
            # extract relevant pet fields
            pets.append(select(pet, {'uuid', 'species', 'breed'}))
            
            # if unique identifiers have been assigned as they should be...
            if policy_id and pet_id:
                # relate the policy to the pet in our many-to-many mapping table
                mapping.append({'policy_id': policy_id, 'pet_id': pet_id})

policies = pd.DataFrame(policies)
pets = pd.DataFrame(pets)
mapping = pd.DataFrame(mapping)

In [280]:
policies

,created_at,account_ref,uuid,price
0,1562171442321,BBM,18de854b-6a0b-4c7e-9c96-affc01770b7a,17891
1,1570205085626,BBM,b14d4923-0593-493a-bb48-e034c8006f61,22318
2,1565956877896,BBM,24748f8e-53e3-4a56-9f18-7ed455c0f966,0
3,1559594266743,BBM,03bceced-9b15-4fea-8160-793c1eeefb53,1432
4,1570629295414,BBM,0b805ab7-bfbf-46f8-873e-a4b495a87ff2,11113
...,...,...,...,...
10521,1569704226574,BBM,cd75fa54-7e63-45c2-b790-e72d8db26fa2,15702
10522,1561584149132,BBM,99a4f752-0bbe-499d-ae88-e9ba02194b3c,230220
10523,1563901572302,BBM,22e28c08-3c23-4035-9158-43edc577a015,0
10524,1556204656381,BBM,d24f41ec-559d-4888-85b0-8ae6ef473ae6,24910


In [281]:
policies[policies.uuid.isnull()]

,created_at,account_ref,uuid,price


In [282]:
mapping

,policy_id,pet_id
0,18de854b-6a0b-4c7e-9c96-affc01770b7a,8aeef6e5-d182-4002-94d9-02f31c357760
1,b14d4923-0593-493a-bb48-e034c8006f61,03a46432-50cc-490d-a4ad-94a0b1442ebe
2,24748f8e-53e3-4a56-9f18-7ed455c0f966,01324f2b-0cfc-4912-a836-19d98681d60c
3,03bceced-9b15-4fea-8160-793c1eeefb53,c40556ed-4713-42ef-99a4-f94bc87ea3ce
4,0b805ab7-bfbf-46f8-873e-a4b495a87ff2,186e8659-5cb7-4fee-8804-619680886155
...,...,...
13212,d24f41ec-559d-4888-85b0-8ae6ef473ae6,b885976b-2e0c-4b8b-9c01-fa7fdf63f5bb
13213,d9559c60-cb8b-4e84-b346-aa00da7a254e,932a0273-184d-4b53-9e66-11371f52a950
13214,d9559c60-cb8b-4e84-b346-aa00da7a254e,15ebf386-d2c0-4cb1-961e-4790bd5ea7b0
13215,d9559c60-cb8b-4e84-b346-aa00da7a254e,09af16fc-0705-46c4-9254-f9faddf4d3c7


As expected, UUIDs are assigned for all 10526 policies, and all 13217 pets. Each pet is covered by exactly one policy.

We now apply some transformations to get our data into an appropriate format. I have chosen to create two tables linked by a foreign key, imitating a relational database.

In [283]:
policies['country'] = policies.account_ref.map(lambda x: 'UK' if x == 'BBM' else 'SWEDEN')

policies.set_index('uuid', inplace=True)
pets.set_index('uuid', inplace=True)
mapping.set_index('pet_id', inplace=True)

pets = pets.join(mapping)

In [284]:
csv_folder = f'{os.getcwd()}/data/csv'

policies.to_csv(f'{csv_folder}/policies.csv', columns=['country', 'created_at', 'price'])
pets.to_csv(f'{csv_folder}/pets.csv')

## 2. Reporting

Let's read our CSVs back in and begin computing metrics.

In [285]:
policies = pd.read_csv(f'{csv_folder}/policies.csv', index_col='uuid')
pets = pd.read_csv(f'{csv_folder}/pets.csv', index_col='uuid')

In [286]:
policies

,country,created_at,price
uuid,,,
18de854b-6a0b-4c7e-9c96-affc01770b7a,UK,1562171442321,17891
b14d4923-0593-493a-bb48-e034c8006f61,UK,1570205085626,22318
24748f8e-53e3-4a56-9f18-7ed455c0f966,UK,1565956877896,0
03bceced-9b15-4fea-8160-793c1eeefb53,UK,1559594266743,1432
0b805ab7-bfbf-46f8-873e-a4b495a87ff2,UK,1570629295414,11113
...,...,...,...
cd75fa54-7e63-45c2-b790-e72d8db26fa2,UK,1569704226574,15702
99a4f752-0bbe-499d-ae88-e9ba02194b3c,UK,1561584149132,230220
22e28c08-3c23-4035-9158-43edc577a015,UK,1563901572302,0


In [287]:
pets

,breed,species,policy_id
uuid,,,
8aeef6e5-d182-4002-94d9-02f31c357760,CHIHUAHUA,DOG,18de854b-6a0b-4c7e-9c96-affc01770b7a
03a46432-50cc-490d-a4ad-94a0b1442ebe,MEDIUM_MONGREL_(10KG-20KG),DOG,b14d4923-0593-493a-bb48-e034c8006f61
01324f2b-0cfc-4912-a836-19d98681d60c,AMERICAN_BULL_DOG,DOG,24748f8e-53e3-4a56-9f18-7ed455c0f966
c40556ed-4713-42ef-99a4-f94bc87ea3ce,GERMAN_SHEPHERD,DOG,03bceced-9b15-4fea-8160-793c1eeefb53
186e8659-5cb7-4fee-8804-619680886155,MOGGIE,CAT,0b805ab7-bfbf-46f8-873e-a4b495a87ff2
...,...,...,...
b885976b-2e0c-4b8b-9c01-fa7fdf63f5bb,WEST_HIGHLAND_WHITE_TERRIER,DOG,d24f41ec-559d-4888-85b0-8ae6ef473ae6
932a0273-184d-4b53-9e66-11371f52a950,MOGGY,CAT,d9559c60-cb8b-4e84-b346-aa00da7a254e
15ebf386-d2c0-4cb1-961e-4790bd5ea7b0,MANX,CAT,d9559c60-cb8b-4e84-b346-aa00da7a254e


In [289]:
policies['purchase_date'] = policies.created_at.map(lambda x: datetime.fromtimestamp(x / 1000))
policies['purchase_month'] = policies.purchase_date.map(lambda x: x.month)

In [290]:
policies

,country,created_at,price,purchase_date,purchase_month
uuid,,,,,
18de854b-6a0b-4c7e-9c96-affc01770b7a,UK,1562171442321,17891,2019-07-03 17:30:42.321,7
b14d4923-0593-493a-bb48-e034c8006f61,UK,1570205085626,22318,2019-10-04 17:04:45.626,10
24748f8e-53e3-4a56-9f18-7ed455c0f966,UK,1565956877896,0,2019-08-16 13:01:17.896,8
03bceced-9b15-4fea-8160-793c1eeefb53,UK,1559594266743,1432,2019-06-03 21:37:46.743,6
0b805ab7-bfbf-46f8-873e-a4b495a87ff2,UK,1570629295414,11113,2019-10-09 14:54:55.414,10
...,...,...,...,...,...
cd75fa54-7e63-45c2-b790-e72d8db26fa2,UK,1569704226574,15702,2019-09-28 21:57:06.574,9
99a4f752-0bbe-499d-ae88-e9ba02194b3c,UK,1561584149132,230220,2019-06-26 22:22:29.132,6
22e28c08-3c23-4035-9158-43edc577a015,UK,1563901572302,0,2019-07-23 18:06:12.302,7
